In [12]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta
import psycopg2
from sqlalchemy import create_engine

# --- Configurações do Banco de Dados RDS ---
DB_HOST = 'db-banco.c07gis2620tg.us-east-1.rds.amazonaws.com'
DB_PORT = '5432'
DB_NAME = 'db_banco'
DB_USER = 'postgres'
DB_PASSWORD = 'XqNuKeQw12345'

# Configuração do Faker para o idioma português do Brasil
fake = Faker('pt_BR')

# Número de registros para cada dataframe
num_clientes = 1000
num_transacoes = 5000
num_emprestimos = 500
num_pagamentos = 2000

# --- Geração dos Dataframes (Código anterior) ---
# ... (Copie aqui o código completo para a geração dos dataframes df_clientes, df_transacoes, df_emprestimos e df_pagamentos) ...
clientes_data = []
fake_unique_cpf = Faker('pt_BR') # Criar um Faker separado para garantir a unicidade do CPF
for _ in range(num_clientes):
    cliente_id = fake.unique.random_number(digits=8)
    nome = fake.name()
    cpf = fake_unique_cpf.unique.cpf() # Usar fake_unique_cpf.unique.cpf()
    data_nascimento = fake.date_of_birth(minimum_age=18, maximum_age=70)
    sexo = random.choice(['Masculino', 'Feminino'])
    endereco = fake.address()
    email = fake.email()
    telefone = fake.phone_number()
    clientes_data.append({
        'cliente_id': cliente_id,
        'nome': nome,
        'cpf': cpf,
        'data_nascimento': data_nascimento,
        'sexo': sexo,
        'endereco': endereco,
        'email': email,
        'telefone': telefone
    })
df_clientes = pd.DataFrame(clientes_data)

transacoes_data = []
tipos_transacao = ['Compra', 'Pagamento', 'Depósito', 'Retirada', 'Transferência']
for _ in range(num_transacoes):
    transacao_id = fake.unique.random_number(digits=10)
    cliente_id = random.choice(df_clientes['cliente_id'].tolist()) # Garante que o ID vem dos clientes gerados
    data_transacao = fake.date_time_between(start_date='-1y', end_date='now')
    tipo_transacao = random.choice(tipos_transacao)
    valor = round(random.uniform(10, 1000), 2)
    descricao = fake.sentence(nb_words=5)
    transacoes_data.append({
        'transacao_id': transacao_id,
        'cliente_id': cliente_id,
        'data_transacao': data_transacao,
        'tipo_transacao': tipo_transacao,
        'valor': valor,
        'descricao': descricao
    })
df_transacoes = pd.DataFrame(transacoes_data)

# --- Dataframe de Empréstimos ---
emprestimos_data = []
tipos_emprestimo = ['Pessoal', 'Automotivo', 'Imobiliário', 'Consignado']
status_emprestimo = ['Aprovado', 'Pendente', 'Pago', 'Em Atraso', 'Negativado']
for _ in range(num_emprestimos):
    emprestimo_id = fake.unique.random_number(digits=9)
    cliente_id = random.choice(df_clientes['cliente_id'].tolist()) # Garante que o ID vem dos clientes gerados
    data_solicitacao = fake.date_time_between(start_date='-2y', end_date='-6mo')
    tipo_emprestimo = random.choice(tipos_emprestimo)
    valor_emprestimo = round(random.uniform(1000, 50000), 2)
    taxa_juros = round(random.uniform(0.01, 0.05), 4)
    prazo_meses = random.randint(12, 60)
    data_aprovacao = fake.date_time_between(start_date=data_solicitacao, end_date='-3mo') if random.random() < 0.8 else None
    status = random.choice(status_emprestimo)
    emprestimos_data.append({
        'emprestimo_id': emprestimo_id,
        'cliente_id': cliente_id,
        'data_solicitacao': data_solicitacao,
        'tipo_emprestimo': tipo_emprestimo,
        'valor_emprestimo': valor_emprestimo,
        'taxa_juros': taxa_juros,
        'prazo_meses': prazo_meses,
        'data_aprovacao': data_aprovacao,
        'status': status
    })
df_emprestimos = pd.DataFrame(emprestimos_data)

# --- Dataframe de Pagamentos ---
def generate_payment_value(emprestimo_row):
    """Função auxiliar para calcular o valor do pagamento"""
    if emprestimo_row['prazo_meses'] != 0:
        base_payment = emprestimo_row['valor_emprestimo'] / emprestimo_row['prazo_meses']
        return round(random.uniform(50, base_payment * 1.2), 2)
    return round(random.uniform(50, emprestimo_row['valor_emprestimo'] * 0.1), 2)

# Filtrar empréstimos aprovados uma única vez
emprestimos_aprovados = df_emprestimos[df_emprestimos['data_aprovacao'].notna()]

# --- Dataframe de Pagamentos (Garantindo Datas Nulas para Atrasos) ---
pagamentos_data = []
emprestimos_aprovados = df_emprestimos[df_emprestimos['data_aprovacao'].notna()]
data_atual_simulacao = datetime(2025, 4, 13, 9, 20, 0) # Usando a data/hora atual

if not emprestimos_aprovados.empty:
    for index, emprestimo in emprestimos_aprovados.iterrows():
        prazo_meses = emprestimo['prazo_meses']
        valor_emprestimo = emprestimo['valor_emprestimo']
        data_aprovacao = emprestimo['data_aprovacao']

        if prazo_meses > 0:
            valor_parcela_teorica = valor_emprestimo / prazo_meses
            for i in range(1, prazo_meses + 1):
                data_vencimento = data_aprovacao + timedelta(days=i * 30) # Aproximação de meses
                data_pagamento_simulada = None
                status_pagamento = 'Pendente'

                # Pagamento em dia ou adiantado
                if random.random() < 0.7:
                    data_pagamento_simulada = data_vencimento - timedelta(days=random.randint(-2, 0))
                    status_pagamento = 'Pago' if data_pagamento_simulada <= data_atual_simulacao else 'Pendente'
                # Se a data de vencimento já passou, o status é 'Em Atraso' e a data de pagamento é nula
                elif data_vencimento < data_atual_simulacao:
                    status_pagamento = 'Em Atraso'
                    data_pagamento_simulada = None
                # Pagamento futuro
                else:
                    data_pagamento_simulada = data_vencimento - timedelta(days=random.randint(-2, 0))
                    status_pagamento = 'Pendente'

                valor_pago = round(random.uniform(valor_parcela_teorica * 0.8, valor_parcela_teorica * 1.2), 2) if status_pagamento == 'Pago' else 0.0

                pagamentos_data.append({
                    'pagamento_id': fake.unique.random_number(digits=10),
                    'emprestimo_id': emprestimo['emprestimo_id'],
                    'numero_parcela': i,
                    'data_vencimento': data_vencimento.date(),
                    'data_pagamento': data_pagamento_simulada.date() if data_pagamento_simulada else None,
                    'valor_pago': valor_pago,
                    'status_pagamento': status_pagamento
                })
        else:
            # Lógica para empréstimos com prazo zero (se houver) - Manter similar com a correção
            data_vencimento = data_aprovacao + timedelta(days=30)
            status_pagamento = 'Em Atraso' if data_vencimento < data_atual_simulacao else 'Pendente'
            data_pagamento_simulada = None if status_pagamento == 'Em Atraso' else (data_vencimento + timedelta(days=random.randint(-5, 10)))
            valor_pago = emprestimo['valor_emprestimo'] if status_pagamento == 'Pago' and data_pagamento_simulada else 0.0
            pagamentos_data.append({
                'pagamento_id': fake.unique.random_number(digits=10),
                'emprestimo_id': emprestimo['emprestimo_id'],
                'numero_parcela': 1,
                'data_vencimento': data_vencimento.date(),
                'data_pagamento': data_pagamento_simulada.date() if data_pagamento_simulada else None,
                'valor_pago': valor_pago,
                'status_pagamento': status_pagamento
            })

df_pagamentos = pd.DataFrame(pagamentos_data)


# --- Função para escrever o dataframe em uma tabela PostgreSQL ---
def write_dataframe_to_rds(dataframe, table_name, db_host, db_port, db_name, db_user, db_password):
    try:
        engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
        dataframe.to_sql(table_name, engine, if_exists='append', index=False)
        print(f"Dataframe '{table_name}' escrito com sucesso no RDS.")
    except Exception as e:
        print(f"Erro ao escrever o dataframe '{table_name}' no RDS: {e}")
    finally:
        if 'engine' in locals():
            engine.dispose()

# --- Escrevendo os dataframes nas tabelas do RDS ---
write_dataframe_to_rds(df_clientes, 'clientes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
write_dataframe_to_rds(df_transacoes, 'transacoes', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
write_dataframe_to_rds(df_emprestimos, 'emprestimos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
write_dataframe_to_rds(df_pagamentos, 'pagamentos', DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)

Dataframe 'clientes' escrito com sucesso no RDS.
Dataframe 'transacoes' escrito com sucesso no RDS.
Dataframe 'emprestimos' escrito com sucesso no RDS.
Dataframe 'pagamentos' escrito com sucesso no RDS.
